In [4]:
import pandas as pd
from google.cloud import storage
import io
import os

# Google Cloud Storage configuration
bucket_name = "adamantine-market-data"
gcs_folder = "market_data/"

# File list in desired order
file_names = [
    'ESUSD_5min.csv', 'NQUSD_5min.csv', 'QQQ_5min.csv', 'SPY_5min.csv', 'USDJPY_5min.csv',
    'ESUSD_15min.csv', 'NQUSD_15min.csv', 'QQQ_15min.csv', 'SPY_15min.csv', 'USDJPY_15min.csv',
    'ESUSD_30min.csv', 'NQUSD_30min.csv', 'QQQ_30min.csv', 'SPY_30min.csv', 'USDJPY_30min.csv',
    'ESUSD_60min.csv', 'NQUSD_60min.csv', 'QQQ_60min.csv', 'SPY_60min.csv', 'USDJPY_60min.csv',
    'ESUSD_240min.csv', 'NQUSD_240min.csv', 'QQQ_240min.csv', 'SPY_240min.csv', 'USDJPY_240min.csv',
    'ESUSD_1440min.csv', 'NQUSD_1440min.csv', 'QQQ_1440min.csv', 'SPY_1440min.csv', 'USDJPY_1440min.csv'
]

# Create storage client
client = storage.Client()
bucket = client.bucket(bucket_name)

summary_data = []
processed_files = 0

for file in file_names:
    blob_path = f"{gcs_folder}{file}"
    blob = bucket.blob(blob_path)

    if not blob.exists(client):
        print(f"Warning: File '{blob_path}' not found in GCS bucket.")
        continue

    try:
        # Read CSV from GCS into pandas DataFrame
        data = blob.download_as_bytes()
        df = pd.read_csv(io.BytesIO(data))

        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=['date'])

        start_date = df['date'].min().strftime("%Y-%m-%d %H:%M")
        end_date = df['date'].max().strftime("%Y-%m-%d %H:%M")
        total_rows = len(df)
        missing_values = df.isnull().sum().sum()

        summary_data.append({
            'File': file,
            'Start Date': start_date,
            'End Date': end_date,
            'Total Observations': total_rows,
            'Missing Values': missing_values
        })
        processed_files += 1

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Check if any files were processed successfully
if processed_files == 0:
    print("\nNo files were processed successfully. Please check if the files exist in GCS.")
    exit()

# Create summary DataFrame
summary_df = pd.DataFrame(summary_data)
ordered_columns = ['File', 'Start Date', 'End Date', 'Total Observations', 'Missing Values']
summary_df = summary_df[ordered_columns]

# Save summary to local disk
output_path = "data_file_summary.xlsx"
summary_df.to_excel(output_path, index=False)

print("\nSummary Table:\n")
print(summary_df.to_string(index=False))

print(f"\nSummary saved to: '{output_path}'")
print(f"Successfully processed {processed_files} out of {len(file_names)} files.")



Summary Table:

              File       Start Date         End Date  Total Observations  Missing Values
    ESUSD_5min.csv 2025-05-06 00:05 2025-05-15 23:40                2193               0
    NQUSD_5min.csv 2025-05-06 00:05 2025-05-15 23:40                2192               0
      QQQ_5min.csv 2025-05-06 09:30 2025-05-15 15:55                 624               0
      SPY_5min.csv 2025-05-06 09:30 2025-05-15 15:55                 624               0
   USDJPY_5min.csv 2025-05-06 00:00 2025-05-15 23:50                2302               0
   ESUSD_15min.csv 2025-04-01 00:00 2025-05-15 23:30                2944               0
   NQUSD_15min.csv 2025-04-01 00:00 2025-05-15 23:30                2941               0
     QQQ_15min.csv 2025-04-01 09:30 2025-05-15 15:45                 832               0
     SPY_15min.csv 2025-04-01 09:30 2025-05-15 15:45                 832               0
  USDJPY_15min.csv 2025-04-01 00:00 2025-05-15 23:45                3161               0
   E

In [12]:
import pandas as pd
from google.cloud import storage
import io
import os

# GCS settings
bucket_name = "adamantine-market-data"
gcs_folder = "market_data/"

# Initialize client
client = storage.Client()
bucket = client.bucket(bucket_name)

# Define exact filename pairs
matched_pairs = [
    ("ESUSD_5min.csv", "NQUSD_5min.csv"),
    ("ESUSD_15min.csv", "NQUSD_15min.csv"),
    ("ESUSD_30min.csv", "NQUSD_30min.csv"),
    ("ESUSD_60min.csv", "NQUSD_60min.csv"),
    ("ESUSD_240min.csv", "NQUSD_240min.csv")
]

# Output folder
output_dir = "matched_timestamp_outputs"
os.makedirs(output_dir, exist_ok=True)

# Summary list
summary = []

# Process each pair
for es_file, nq_file in matched_pairs:
    try:
        # Load from GCS
        es_blob = bucket.blob(f"{gcs_folder}{es_file}")
        nq_blob = bucket.blob(f"{gcs_folder}{nq_file}")
        es_df = pd.read_csv(io.BytesIO(es_blob.download_as_bytes()))
        nq_df = pd.read_csv(io.BytesIO(nq_blob.download_as_bytes()))

        # Parse dates
        es_df['date'] = pd.to_datetime(es_df['date'], errors='coerce')
        nq_df['date'] = pd.to_datetime(nq_df['date'], errors='coerce')
        es_df = es_df.dropna(subset=['date'])
        nq_df = nq_df.dropna(subset=['date'])

        # Match timestamps
        matched_df = pd.merge(es_df[['date']], nq_df[['date']], on='date', how='inner')
        only_in_es = es_df[~es_df['date'].isin(nq_df['date'])]
        only_in_nq = nq_df[~nq_df['date'].isin(es_df['date'])]

        # Save matched dates
        out_file = f"matched_{es_file.replace('.csv','')}_{nq_file.replace('.csv','')}.xlsx"
        matched_df.to_excel(os.path.join(output_dir, out_file), index=False)

        # Record summary
        summary.append({
            "Pair": f"{es_file} & {nq_file}",
            "Matched": len(matched_df),
            "Only in ES": len(only_in_es),
            "Only in NQ": len(only_in_nq)
        })

    except Exception as e:
        summary.append({
            "Pair": f"{es_file} & {nq_file}",
            "Matched": "Error",
            "Only in ES": "Error",
            "Only in NQ": str(e)
        })

# Final summary table
summary_df = pd.DataFrame(summary)
summary_path = os.path.join(output_dir, "timestamp_match_summary.xlsx")
#summary_df.to_excel(summary_path, index=False)

# Print
print("\n Timestamp Match Summary:\n")
print(summary_df.to_string(index=False))
print(f"\n Summary saved at: {summary_path}")



 Timestamp Match Summary:

                               Pair  Matched  Only in ES  Only in NQ
    ESUSD_5min.csv & NQUSD_5min.csv     2188           5           4
  ESUSD_15min.csv & NQUSD_15min.csv     2940           4           1
  ESUSD_30min.csv & NQUSD_30min.csv      967           0           0
  ESUSD_60min.csv & NQUSD_60min.csv     1443           4           0
ESUSD_240min.csv & NQUSD_240min.csv      750           9           1

 Summary saved at: matched_timestamp_outputs/timestamp_match_summary.xlsx


In [15]:
#Descriptive Statistics
import pandas as pd
import io
from scipy.stats import skew, kurtosis
from google.cloud import storage

# Define GCS connection
bucket_name = "adamantine-market-data"
gcs_folder = "market_data/"
client = storage.Client()
bucket = client.bucket(bucket_name)

# Exact file pairs
matched_pairs = [
    ("ESUSD_5min.csv", "NQUSD_5min.csv"),
    ("ESUSD_15min.csv", "NQUSD_15min.csv"),
    ("ESUSD_30min.csv", "NQUSD_30min.csv"),
    ("ESUSD_60min.csv", "NQUSD_60min.csv"),
    ("ESUSD_240min.csv", "NQUSD_240min.csv")
]

# Collect stats
all_stats = []

for es_file, nq_file in matched_pairs:
    for label, file in zip(["ESUSD", "NQUSD"], [es_file, nq_file]):
        try:
            blob = bucket.blob(f"{gcs_folder}{file}")
            df = pd.read_csv(io.BytesIO(blob.download_as_bytes()))

            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            df = df.dropna(subset=['date'])

            for col in ['open', 'high', 'low', 'close', 'volume']:
                if col in df.columns:
                    series = pd.to_numeric(df[col], errors='coerce').dropna()
                    all_stats.append({
                        "File": f"{label}_{file}",
                        "Metric": col,
                        "Mean": series.mean(),
                        "Std Dev": series.std(),
                        "Min": series.min(),
                        "Max": series.max(),
                        "Skewness": skew(series),
                        "Kurtosis": kurtosis(series)
                    })

        except Exception as e:
            all_stats.append({
                "File": f"{label}_{file}",
                "Metric": "ERROR",
                "Mean": "N/A",
                "Std Dev": "N/A",
                "Min": "N/A",
                "Max": "N/A",
                "Skewness": "N/A",
                "Kurtosis": str(e)
            })

# Final summary DataFrame
stats_df = pd.DataFrame(all_stats)
#stats_df.to_excel("descriptive_stats_summary.xlsx", index=False)

# Save and print
print("\n📊 Descriptive Statistics:\n")
print(stats_df.to_string(index=False))


📊 Descriptive Statistics:

                  File Metric          Mean       Std Dev      Min        Max  Skewness  Kurtosis
  ESUSD_ESUSD_5min.csv   open   5779.349179    110.258131  5600.25    5942.50  0.028773 -1.694174
  ESUSD_ESUSD_5min.csv   high   5781.519494    110.023273  5608.75    5944.50  0.030029 -1.697802
  ESUSD_ESUSD_5min.csv    low   5777.115253    110.593801  5597.75    5940.25  0.027375 -1.690981
  ESUSD_ESUSD_5min.csv  close   5779.392841    110.324413  5601.75    5942.75  0.028019 -1.694756
  ESUSD_ESUSD_5min.csv volume   3481.369813   6918.328326     0.00  106671.00  5.191309 47.767522
  NQUSD_NQUSD_5min.csv   open  20622.662568    578.866960 19703.75   21523.00  0.106394 -1.636983
  NQUSD_NQUSD_5min.csv   high  20632.346943    577.872140 19731.00   21529.75  0.107561 -1.640202
  NQUSD_NQUSD_5min.csv    low  20612.782276    580.262929 19678.75   21512.25  0.103955 -1.633860
  NQUSD_NQUSD_5min.csv  close  20623.024407    579.209109 19703.75   21523.50  0.104922 -1

#### Why focus on 15-minute, 30-minute, and 1-hour prediction horizons?
| Key Trait              | Ideal Condition                     | Why It Matters                       |
| ---------------------- | ----------------------------------- | ------------------------------------ |
| **Skewness**           | Close to 0                          | Reduces bias in prediction           |
| **Kurtosis**           | Close to 0–1                        | Limits outlier influence             |
| **Volume stability**   | Moderate SD and lower kurtosis/skew | Reduces false spikes, smoother model |
| **Standard Deviation** | Not excessively high                | Reflects better signal quality       |

🔘 5-Minute: ❌ Too noisy — avoid.
Price Std Dev: 110.32 (ES), 579.21 (NQ) ❗ | Skewness: +0.028 (ES), +0.105 (NQ) ✔ | Volume skewness: 5.19 (ES), 2.93 (NQ) ❗
Kurtosis: Price –1.695 (ES), –1.638 (NQ) ✔ | Volume kurtosis: 47.77 (ES), 13.18 (NQ) ❗
Interpretation: Stable prices but high-frequency noise and extreme volume outliers make this timeframe unsuitable for reliable spread modeling.

🔘 15-Minute: ✅ Best all-around choice for signal generation.
Price Std Dev: 237.65 (ES), 1077.79 (NQ) ✔ | Skewness: –0.402 (ES), –0.072 (NQ) ✔ | Volume skewness: 3.00 (ES), 2.41 (NQ) ⚠
Kurtosis: Price –0.136 (ES), –0.234 (NQ) ✔ | Volume kurtosis: 14.16 (ES), 6.80 (NQ) ⚠
Interpretation: Smooth and symmetric price dynamics offer a strong foundation for Z-score prediction; volume extremes are secondary.

🔘 30-Minute: ✅ Strong supporting timeframe.
Price Std Dev: 199.36 (ES), 955.34 (NQ) ✔ | Skewness: –0.182 (ES), –0.015 (NQ) ✔ | Volume skewness: 2.14 (ES), 1.75 (NQ) ✔
Kurtosis: Price –0.681 (ES), –0.704 (NQ) ✔ | Volume kurtosis: 5.51 (ES), 2.75 (NQ) ✔
Interpretation: Well-behaved price distributions and lower noise make this timeframe ideal for validating Z-score-based signals.

🔘 60-Minute: ✅ Ideal for strategic overlays, not quick exits.
Price Std Dev: 265.57 (ES), 1177.80 (NQ) ✔ | Skewness: –0.345 (ES), +0.007 (NQ) ✔ | Volume skewness: 1.84 (ES), 1.67 (NQ) ✔
Kurtosis: Price +0.132 (ES), –0.013 (NQ) ✔ | Volume kurtosis: 3.32 (ES), 1.98 (NQ) ✔
Interpretation: Stable, near-normal distribution supports macro-level regime detection; ideal for risk filters or directional overlays.

🔘 240-Minute: ❌ Use only for macro filters — avoid for prediction.
Price Std Dev: 268.85 (ES), N/A (NQ close missing) ❗ | Skewness: –1.090 (ES), –0.850 (NQ) ❗ | Volume skewness: 1.42 (ES), 1.28 (NQ) ✔
Kurtosis: Price +0.651 (ES), +0.105 (NQ) ✔ | Volume kurtosis: 1.45 (ES), 0.58 (NQ) ✔
Interpretation: Negatively skewed and sluggish behavior reflects directional bias; not suitable for short-term modeling or timely exits.


In [16]:
import pandas as pd
import io
from google.cloud import storage

# ---------------------------
# GCS Setup
# ---------------------------
bucket_name = "adamantine-market-data"
gcs_folder = "market_data/"
client = storage.Client()
bucket = client.bucket(bucket_name)

# ---------------------------
# Define exact matched pairs
# ---------------------------
matched_pairs = [
    ("ESUSD_5min.csv", "NQUSD_5min.csv"),
    ("ESUSD_15min.csv", "NQUSD_15min.csv"),
    ("ESUSD_30min.csv", "NQUSD_30min.csv"),
    ("ESUSD_60min.csv", "NQUSD_60min.csv"),
    ("ESUSD_240min.csv", "NQUSD_240min.csv")
]

# ---------------------------
# Analyze correlation
# ---------------------------
correlation_stats = []

for es_file, nq_file in matched_pairs:
    try:
        # Read ESUSD file from GCS
        es_blob = bucket.blob(f"{gcs_folder}{es_file}")
        es_data = es_blob.download_as_bytes()
        es_df = pd.read_csv(io.BytesIO(es_data))
        es_df['date'] = pd.to_datetime(es_df['date'], errors='coerce')
        es_df = es_df.dropna(subset=['date', 'close'])

        # Read NQUSD file from GCS
        nq_blob = bucket.blob(f"{gcs_folder}{nq_file}")
        nq_data = nq_blob.download_as_bytes()
        nq_df = pd.read_csv(io.BytesIO(nq_data))
        nq_df['date'] = pd.to_datetime(nq_df['date'], errors='coerce')
        nq_df = nq_df.dropna(subset=['date', 'close'])

        # Merge on date and calculate correlation
        merged_df = pd.merge(
            es_df[['date', 'close']], 
            nq_df[['date', 'close']], 
            on='date', 
            suffixes=('_es', '_nq')
        ).dropna()

        # Pearson correlation between ES and NQ closing prices
        correlation = merged_df['close_es'].corr(merged_df['close_nq'])

        correlation_stats.append({
            "Pair": f"{es_file} & {nq_file}",
            "Correlation": correlation,
            "Observations": len(merged_df)
        })

    except Exception as e:
        correlation_stats.append({
            "Pair": f"{es_file} & {nq_file}",
            "Correlation": "Error",
            "Observations": 0,
            "Error": str(e)
        })

# ---------------------------
# Print or save results
# ---------------------------
correlation_df = pd.DataFrame(correlation_stats)
print("\n📈 Correlation Summary:\n")
print(correlation_df.to_string(index=False))



📈 Correlation Summary:

                               Pair  Correlation  Observations
    ESUSD_5min.csv & NQUSD_5min.csv     0.994750          2188
  ESUSD_15min.csv & NQUSD_15min.csv     0.988819          2940
  ESUSD_30min.csv & NQUSD_30min.csv     0.997655           967
  ESUSD_60min.csv & NQUSD_60min.csv     0.984807          1443
ESUSD_240min.csv & NQUSD_240min.csv     0.981284           750
